# Exercise 2 - Transfer Learning VGG16 

We will first investigate how to train the model via transfer learning, after this we will cleanup and encapsulate the methodolgy in a *MnistClassifier* class that allows for easy training and predicting.

In [3]:
import numpy as np
import keras
from keras.layers import Dense
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import img_to_array, array_to_img
import tensorflow.image as tf_image
import matplotlib.pyplot as plt

Load in data, we will limit the dataset sizes for examples sake.

In [4]:
TRAIN_SIZE = 1000
TEST_SIZE = 5000

In [23]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [6]:
model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

Preprocess to rgb

In [25]:
def preprocess(images):
    """ Preprocess grey scale image to rgb and resizes
    
    Args:
        images (np.array): Array of grey scale image
        
    Returns:
        np.array: Array of converted images
    """
    image_rgb = np.dstack((images, np.zeros_like(images), np.zeros_like(images)))
    image_rgb = image_rgb.reshape(image_rgb.shape[0], 28, 28, 3)
    image_rgb = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in image_rgb])
    return image_rgb
    
x_train_rgb = preprocess(x_train)
y_train_oh = to_categorical(y_train)

x_test_rgb = preprocess(x_test)
y_test_oh = to_categorical(y_test)

(60000, 10)


Extract features from VGG16

In [8]:
x_train_rgb.shape
train_features = model.predict(x_train_rgb[:TRAIN_SIZE])
train_features.shape

(1000, 1, 1, 512)

Train simple mlp on the extracted features

In [9]:
from keras.layers import Dense, Activation
from keras.models import Model
from keras import models
from keras import layers
from keras import optimizers

shape = np.array(train_features.shape)
train_features = np.reshape(train_features, (shape[0], shape[-1]))
print(train_features.shape)

model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_dim=512))
model.add(Dense(500, activation='sigmoid'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

# fitting the model 

model.fit(train_features, y_train_oh[:TRAIN_SIZE], epochs=20, batch_size=128)

(1000, 512)
Epoch 1/20
1000/1000 [==============================] - 1s 682us/step - loss: 1.8952 - accuracy: 0.3880
Epoch 2/20
1000/1000 [==============================] - 0s 137us/step - loss: 0.9573 - accuracy: 0.7330
Epoch 3/20
1000/1000 [==============================] - 0s 137us/step - loss: 0.6615 - accuracy: 0.8020
Epoch 4/20
1000/1000 [==============================] - 0s 137us/step - loss: 0.4821 - accuracy: 0.8650
Epoch 5/20
1000/1000 [==============================] - 0s 136us/step - loss: 0.3615 - accuracy: 0.9030
Epoch 6/20
1000/1000 [==============================] - 0s 129us/step - loss: 0.2832 - accuracy: 0.9410
Epoch 7/20
1000/1000 [==============================] - 0s 116us/step - loss: 0.2066 - accuracy: 0.9580
Epoch 8/20
1000/1000 [==============================] - 0s 122us/step - loss: 0.1548 - accuracy: 0.9790
Epoch 9/20
1000/1000 [==============================] - 0s 135us/step - loss: 0.1176 - accuracy: 0.9850
Epoch 10/20
1000/1000 [=============================

Lets tidy this up and build it as a wrapper class

In [61]:
class MnistClassifier():
    """ Transer learning wrapper class for classifying the Mnist data set.
    """
    IMAGE_HEIGHT_ = 48
    IMAGE_WIDTH_ = 48
    
    def __init__(self):
        self.vgg16 = VGG16(weights='imagenet', include_top=False, 
                           input_shape=(self.IMAGE_HEIGHT_, self.IMAGE_WIDTH_, 3))
        model = models.Sequential()
        model.add(layers.Dense(512, activation='relu', input_dim=512))
        model.add(Dense(500, activation='sigmoid'))
        # We will add the final dense layer one we determin the number of outputs
        self.output_model = model
        self.class_map = {} # Map classes to categories 
        self.trained = False
    
    def preprocess(self, images):
        """ Preprocess grey scale image to rgb and resizes

        Args:
            images (np.array): Array of grey scale image

        Returns:
            np.array: Array of converted images
        """     
        image_rgb = np.dstack((images, np.zeros_like(images), np.zeros_like(images)))
        image_rgb = image_rgb.reshape(image_rgb.shape[0], 28, 28, 3)
        image_rgb = np.asarray([img_to_array(array_to_img(im, scale=False).resize((self.IMAGE_HEIGHT_, self.IMAGE_WIDTH_))) for im in image_rgb])
        return image_rgb
    
    def create_class_map(self, targets):
        """ Store the one-hot encoding as a map.
        """
        classes = list(set(targets))
        self.class_map = {}
        for i in range(len(classes)):
            enc = np.zeros(len(classes))
            enc[i] = 1
            self.class_map[classes[i]] = enc
        return self.class_map
    
    def encode(self, y):
        """ Encode and store encoding as a map.
        """
        cm = self.create_class_map(y)
        y_one_hot = [cm[y_i] for y_i in y]
        return np.array(y_one_hot)
    
    def extract_vgg16_ft(self, images):
        """ Extract the features from vgg16.
        """
        fts = self.vgg16.predict(images)
        fts = np.reshape(fts, (fts.shape[0], fts.shape[-1]))
        return fts
            
    def train(self, x, y, epochs=20):
        """ Train the model.
        
        This encapsulates the full pipe, it preprocess the images and extract 
        a mapping for class labels to one-hot-encoding.
        """
        x_rgb = self.preprocess(x)
        y_one_hot = self.encode(y)
        # add the dense output layer

        self.output_model.add(layers.Dense(len(self.class_map), activation='softmax'))
        self.output_model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
        self.output_model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
        
        fts = self.extract_vgg16_ft(x_rgb)
        self.output_model.fit(fts, y_one_hot, epochs=epochs, batch_size=128)
        self.trained = True
        
    def predict(self, images):
        """ Predict classes.
        """
        assert(self.trained)
        img_rgb = self.preprocess(images)
        fts = self.extract_vgg16_ft(img_rgb)
        y_ = self.output_model.predict(fts)
        # Reverse the class map to get the predicted class label
        rvs_map = {np.argmax(v):k for k, v in self.class_map.items()}
        return np.array([rvs_map[np.argmax(y_i)] for y_i in y_])
        

Train on the first 1000 samples

In [62]:
cls = MnistClassifier()
cls.train(x_train[:1000], y_train[:1000])

Epoch 1/20
1000/1000 [==============================] - 1s 847us/step - loss: 1.9175 - accuracy: 0.3700
Epoch 2/20
1000/1000 [==============================] - 0s 128us/step - loss: 0.9890 - accuracy: 0.7210
Epoch 3/20
1000/1000 [==============================] - 0s 128us/step - loss: 0.6737 - accuracy: 0.8010
Epoch 4/20
1000/1000 [==============================] - 0s 124us/step - loss: 0.4786 - accuracy: 0.8710
Epoch 5/20
1000/1000 [==============================] - 0s 127us/step - loss: 0.3676 - accuracy: 0.9060
Epoch 6/20
1000/1000 [==============================] - 0s 126us/step - loss: 0.2857 - accuracy: 0.9360
Epoch 7/20
1000/1000 [==============================] - 0s 123us/step - loss: 0.2187 - accuracy: 0.9590
Epoch 8/20
1000/1000 [==============================] - 0s 121us/step - loss: 0.1695 - accuracy: 0.9770
Epoch 9/20
1000/1000 [==============================] - 0s 122us/step - loss: 0.1303 - accuracy: 0.9850
Epoch 10/20
1000/1000 [==============================] - 0s 122u

Example test of accuracy on the first 20 test samples

In [63]:
pred = cls.predict(x_test[:20])
print(pred)
print(np.array(y_test[:20]))
acc = sum(np.array(y_test[:20]) == pred)/len(pred)
f'Accuracy: {acc}'

[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]


'Accuracy: 1.0'